In [1]:
!pip install crewai openai gradio transformers accelerate --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.7/366.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.9 MB/s eta 0:

In [3]:
from crewai import Crew, Agent, Task
import gradio as gr
from transformers import pipeline

# Sentiment model pipeline (DistilBERT)
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [4]:
mood_agent = Agent(
    role="Mood Analyzer",
    goal="Analyze the user's emotional tone from their input text",
    backstory="Expert in NLP and psychology, specialized in sentiment detection.",
    allow_delegation=False,
    verbose=True
)

companion_agent = Agent(
    role="Companion",
    goal="Provide empathetic and encouraging support based on the user's mood.",
    backstory="A warm, compassionate AI trained in conversational support.",
    allow_delegation=False,
    verbose=True
)

selfcare_agent = Agent(
    role="Self-Care Recommender",
    goal="Suggest personalized wellness activities depending on the user's current mood.",
    backstory="Mental health AI coach trained to suggest self-care tasks based on emotion.",
    allow_delegation=False,
    verbose=True
)

In [5]:
def mental_health_bot(user_input):
    sentiment = sentiment_pipeline(user_input)[0]
    label = sentiment['label']
    score = sentiment['score']

    if label == 'POSITIVE':
        mood = 'happy'
    elif label == 'NEGATIVE':
        mood = 'sad'
    else:
        mood = 'neutral'

    task_mood = Task(
        description=f"Analyze this user message: '{user_input}' and confirm their emotional state.",
        expected_output=f"The user seems to be feeling {mood}.",
        agent=mood_agent
    )

    task_companion = Task(
        description=f"The user is feeling {mood}. Give a comforting and empathetic reply.",
        expected_output="A short message of encouragement.",
        agent=companion_agent
    )

    task_selfcare = Task(
        description=f"The user is feeling {mood}. Suggest 2-3 specific self-care activities for them.",
        expected_output="List of activities like journaling, walking, etc.",
        agent=selfcare_agent
    )

    crew = Crew(
        agents=[mood_agent, companion_agent, selfcare_agent],
        tasks=[task_mood, task_companion, task_selfcare],
        verbose=True
    )

    try:
        import asyncio
        results = asyncio.run(crew.kickoff())
    except Exception as e:
        return mood, f"{score:.2f}", f"Agent error: {str(e)}", "—"

    if "---" in results:
        parts = results.split("---")
        return mood.capitalize(), f"{score:.2f}", parts[1].strip(), parts[2].strip()
    else:
        return mood.capitalize(), f"{score:.2f}", "Error in companion", "Error in self-care"


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Mental Health Support Bot (CrewAI powered)")
    inp = gr.Textbox(lines=3, placeholder="Type how you're feeling...")
    mood_out = gr.Text(label="Detected Mood")
    score_out = gr.Text(label="Confidence")
    comp_out = gr.Textbox(label="Companion Response")
    care_out = gr.Textbox(label="Self-Care Suggestions")
    btn = gr.Button("Analyze Mood")

    btn.click(mental_health_bot, inputs=inp, outputs=[mood_out, score_out, comp_out, care_out])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://86ff0e018a212f2452.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2b1fe401-759b-4cf2-a39c-85c4d389743a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Mood Analyzer                                                                                           │
│                                                                                                                 │
│  Task: Analyze this user message: 'I am feeling very anxious about placements , but i have hope that i will do  │
│  good  in the placements with the help from the my internship experience.                                       │
│  ' and confirm their emotional state.                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Mood Analyzer                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The user is expressing a complex emotional state characterized by anxiety and hope. They feel "very anxious"   │
│  about their placements, indicating a sense of worry and unease regarding the outcome of this significant       │
│  event. However, this anxiety is tempered by a sense of hope that they will perform well, particularly because  │
│  they are relying on their internship experience as a source of confidence and support. This blend of anxiety   │
│  and hope suggests that while the user is currently feeling a higher level of stress, they also possess a       │
│  positive outlook on their future performance, indicating resilience and a willingness to overcome challenges.  │
│  Therefore, the user's emotional state reflects both apprehension and optimism, portraying a nuanced            │
│  perspective on their situation.                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Self-Care Recommender                                                                                   │
│                                                                                                                 │
│  Task: The user is feeling happy. Suggest 2-3 specific self-care activities for them.                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Self-Care Recommender                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Here are three self-care activities that can enhance your happiness and prepare you for your placements:       │
│                                                                                                                 │
│  1. **Gratitude Journaling**: Take a few moments to reflect on what makes you happy, including the skills       │
│  you've gained from your internship. Write down at least three things you're grateful for today. This practice  │
│  not only reinforces your positive emotions but also helps you build resilience against anxiety by focusing on  │
│  the positive aspects of your life.                                                                             │
│                                                                                                                 │
│  2. **Nature Walk**: Spend some time outdoors, breathing in the fresh air and enjoying the beauty of nature.    │
│  Going for a walk in a park or your favorite outdoor spot can elevate your mood even further and help clear     │
│  your mind. As you walk, think about your upcoming placements and visualize yourself succeeding, drawing on     │
│  the confidence from your experience.                                                                           │
│                                                                                                                 │
│  3. **Creative Expression**: Engage in a fun, creative activity that lets your happiness shine! Whether it's    │
│  painting, crafting, or playing music, set aside time to express yourself. This will not only amplify your      │
│  feelings of joy but also serve as a wonderful distraction from any anxious thoughts about your placements.     │
│                                                                                                                 │
│  Embrace these activities and let your happiness fuel your confidence as you prepare for your upcoming          │
│  challenges!                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: aaff2b8d-ed9b-4f43-82aa-24b244a8802c                                                                     │
│  Agent: Self-Care Recommender                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2b1fe401-759b-4cf2-a39c-85c4d389743a                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Here are three self-care activities that can enhance your happiness and prepare you for your     │
│  placements:                                                                                                    │
│                                                                                                                 │
│  1. **Gratitude Journaling**: Take a few moments to reflect on what makes you happy, including the skills       │
│  you've gained from your internship. Write down at least three things you're grateful for today. This practice  │
│  not only reinforces your positive emotions but also helps you build resilience against anxiety by focusing on  │
│  the positive aspects of your life.                                                                             │
│                                                                                                                 │
│  2. **Nature Walk**: Spend some time outdoors, breathing in the fresh air and enjoying the beauty of nature.    │
│  Going for a walk in a park or your favorite outdoor spot can elevate your mood even further and help clear     │
│  your mind. As you walk, think about your upcoming placements and visualize yourself succeeding, drawing on     │
│  the confidence from your experience.                                                                           │
│                                                                                                                 │
│  3. **Creative Expression**: Engage in a fun, creative activity that lets your happiness shine! Whether it's    │
│  painting, crafting, or playing music, set aside time to express yourself. This will not only amplify your      │
│  feelings of joy but also serve as a wonderful distraction from any anxious thoughts about your placements.     │
│                                                                                                                 │
│  Embrace these activities and let your happiness fuel your confidence as you prepare for your upcoming          │
│  challenges!                                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯